In [1]:
import pickle
import nest_asyncio
from datasets import load_dataset

import re

import json

from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification, AutoModelForCausalLM
import networkx as nx
import spacy
from itertools import combinations

from llama_index.core import (
    KnowledgeGraphIndex,
    StorageContext,
    Document,
    Settings,
    SimpleDirectoryReader
)
from llama_index.core import KnowledgeGraphIndex, StorageContext
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.schema import TextNode, NodeRelationship
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor
from llama_index.core.query_engine import KnowledgeGraphQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

from llama_index.llms.llama_cpp import LlamaCPP


/mloscratch/homes/ordonnea/conda/envs/graph_rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mloscratch/homes/ordonnea/conda/envs/graph_rag/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_url" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/mloscratch/homes/ordonnea/conda/envs/graph_rag/lib/python3.12/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_path" in LlamaCPP has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/mloscratch/homes/ordonnea/conda/envs/graph_rag/lib/python3.12/site-packages/pydant

In [2]:
model_url = "https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF/resolve/main/Meta-Llama-3-8B-Instruct.Q4_0.gguf"
llm_llama3 = LlamaCPP(
    model_url=model_url,
    model_path=None,
    temperature=0.1,
    max_new_tokens=512,
    context_window=10000,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": 2},
    verbose=True,
)

Settings.llm = llm_llama3
Settings.chunk_size = 512

embed_model = HuggingFaceEmbedding(model_name = "dunzhang/stella_en_400M_v5", device = "cuda", trust_remote_code=True, embed_batch_size=20)

nlp = spacy.load("en_core_web_sm")

total size (MB): 4661.21


4446it [00:20, 213.70it/s]                          
llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from /tmp/llama_index/models/Meta-Llama-3-8B-Instruct.Q4_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Models
llama_model_loader: - kv   3:                         general.size_label str              = 8.0B
llama_model_loader: - kv   4:                            general.license str              = llama3
llama_model_loader: - kv   5:                               general.tags arr[str,6]       = ["facebook", "meta", "pytorch", "llam...
llama_model_loader: - kv   6:                          ge

In [ ]:
def load_wikitext(split="train"):
    dataset = load_dataset("Salesforce/wikitext", "wikitext-103-raw-v1")
    
    documents = []
    current_doc = ""
    current_title = ""
    
    for item in dataset[split]:
        text = item['text']
        if text.startswith(" = ") and text.endswith(" = \n") and not text.startswith(" = = "):
            if current_doc != "":
                documents.append(Document(text=current_doc, metadata={"title": current_title}))
            current_title = text
            current_doc = ""
        else:
           current_doc += text + "\n"
    
    if current_doc:
        documents.append(Document(text=current_doc, metadata={"title": current_title}))
    
    return documents

wikitext_docs = load_wikitext()
print(f"Loaded {len(wikitext_docs)} documents from WikiText")

In [4]:
def clean_text(text):
    text = re.sub(r'@-@', '-', text)  # Replace @-@ with actual hyphens
    text = re.sub(r'@\w+', '', text)  # Remove other @ annotations
    text = re.sub(r'\n+', '\n', text)  # Replace multiple newlines with a single one
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'\s+\.', '.', text)  # Remove spaces before periods
    text = re.sub(r'\s+,', ',', text)  # Remove spaces after commas
    text = re.sub(r'\s+:', ':', text)  # Remove spaces before colons
    text = re.sub(r'\s+\'', '\'', text)  # Remove spaces before '
    return text.strip()

for doc in wikitext_docs[:10]:
    doc.text = clean_text(doc.text)

In [5]:
for doc in wikitext_docs[:10]:
    print(doc.metadata["title"])
    print(doc.text)
    print()




 = Valkyria Chronicles III = 

Senjō no Valkyria 3: Unrecorded Chronicles ( Japanese: 戦場のヴァルキュリア3 ,lit. Valkyria of the Battlefield 3 ) ,commonly referred to as Valkyria Chronicles III outside Japan ,is a tactical role - playing video game developed by Sega and Media.Vision for the PlayStation Portable. Released in January 2011 in Japan ,it is the third game in the Valkyria series. Employing the same fusion of tactical and real - time gameplay as its predecessors ,the story runs parallel to the first game and follows the " Nameless " ,a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " Calamaty Raven ". The game began development in 2010 ,carrying over a large portion of the work done on Valkyria Chronicles II. While it retained the standard features of the series ,it also underwent multiple adjustments ,such as making the game more forgiving for series newcomers. Character de

In [ ]:
def lowest_common_ancestor(token1, token2):
    path1 = list(token1.ancestors)
    path2 = list(token2.ancestors)
    
    if token1 in path2:
        return token1
    if token2 in path1:
        return token2

    for ancestor in path1:
        if ancestor in path2:
            return ancestor

    return None

def find_root_verb(subject, object, sent):
    subject_token = subject.root
    object_token = object.root
    
    if subject_token.i > object_token.i:
        subject_token, object_token = object_token, subject_token

    lca = lowest_common_ancestor(subject_token, object_token)
    
    if lca.pos_ == "VERB":
        return lca
    
    while lca.head != lca:
        if lca.pos_ == "VERB":
            return lca
        lca = lca.head
    
    return None

def extract_triplets(text):
    doc = nlp(text)
    triplets = []

    for sent in doc.sents:
        entity_tokens = [ent for ent in sent.ents]
        
        # Extract subject-object pairs
        for subject, object in combinations(entity_tokens, 2):
            # Find the root verb between subject and object
            root_verb = find_root_verb(subject, object, sent)
            if root_verb:
                triplets.append((subject.text, root_verb.lemma_, object.text))

        # Extract noun-chunks based triplets
        noun_chunks = list(sent.noun_chunks)
        for chunk in noun_chunks:
            if chunk.root.dep_ in ["nsubj", "nsubjpass"]:
                verb = chunk.root.head
                for child in verb.children:
                    if child.dep_ in ["dobj", "attr", "prep"]:
                        triplets.append((chunk.text, verb.lemma_, child.text))

    return triplets

def build_knowledge_graph(documents):
    G = nx.DiGraph()
    
    for doc in documents:
        triplets = extract_triplets(doc.text)
        for subject, predicate, object in triplets:
            G.add_edge(subject, object, relation=predicate)
    
    return G

In [ ]:
knowledge_graph = build_knowledge_graph(wikitext_docs)

print(f"Knowledge Graph built with {knowledge_graph.number_of_nodes()} nodes and {knowledge_graph.number_of_edges()} edges")

# Display some triplets
print("\nSample triplets:")
for edge in list(knowledge_graph.edges(data=True))[:10]:
    print(f"{edge[0]} -- {edge[2]['relation']} --> {edge[1]}")

# Basic analysis
print(f"\nMost common relations:")
relation_counts = Counter([edge[2]['relation'] for edge in knowledge_graph.edges(data=True)])
for relation, count in relation_counts.most_common(5):
    print(f"{relation}: {count}")

print(f"\nMost connected entities:")
degree_centrality = nx.degree_centrality(knowledge_graph)
for entity, centrality in sorted(degree_centrality.items(), key=lambda x: x[1], reverse=True)[:5]:
    print(f"{entity}: {centrality:.4f}")

In [6]:
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

index = KnowledgeGraphIndex.from_documents(
    wikitext_docs[:10],
    storage_context=storage_context,
    max_triplets_per_chunk=10,
    include_embeddings=True,
    llm=llm_llama3,
    embed_model=embed_model,
    show_progress=True
)

Processing nodes:   0%|          | 0/66 [00:00<?, ?it/s]
llama_print_timings:        load time =    9485.70 ms
llama_print_timings:      sample time =      44.36 ms /   512 runs   (    0.09 ms per token, 11542.97 tokens per second)
llama_print_timings: prompt eval time =    9485.39 ms /   114 tokens (   83.21 ms per token,    12.02 tokens per second)
llama_print_timings:        eval time =   29853.05 ms /   511 runs   (   58.42 ms per token,    17.12 tokens per second)
llama_print_timings:       total time =   40134.50 ms /   625 tokens
Processing nodes:   2%|▏         | 1/66 [00:40<43:56, 40.56s/it]Llama.generate: 110 prefix-match hit, remaining 465 prompt tokens to eval

llama_print_timings:        load time =    9485.70 ms
llama_print_timings:      sample time =      36.53 ms /   408 runs   (    0.09 ms per token, 11170.13 tokens per second)
llama_print_timings: prompt eval time =   12276.08 ms /   465 tokens (   26.40 ms per token,    37.88 tokens per second)
llama_print_timings:  

In [8]:
query_engine = index.as_query_engine(
    include_text=True,
    response_mode="tree_summarize",
    embedding_mode="hybrid",
    similarity_top_k=5,
)

def rag_with_knowledge_graph(query: str):
    retrieval_results = query_engine.retrieve(query)
    
    context = "\n".join([node.get_content() for node in retrieval_results])
    
    prompt = f"""
    Based on the following information from a knowledge graph:
    {context}
    
    Please answer the following question:
    {query}
    
    If the information is not sufficient to answer the question, please say so.
    """
    
    response = llm_llama3.complete(prompt)
    
    return response.text

def multi_hop_rag(query: str, max_hops: int = 3):
    context = ""
    for hop in range(max_hops):
        retrieval_results = query_engine.retrieve(query)
        hop_context = "\n".join([node.get_content() for node in retrieval_results])
        context += f"\nHop {hop + 1}:\n{hop_context}\n"
        
        prompt = f"""
        Based on the following information from a knowledge graph:
        {context}
        
        Please answer the following question:
        {query}
        
        If you can answer the question, do so. If not, what additional information do you need?
        """
        
        response = llm_llama3.complete(prompt)
        
        if "additional information" not in response.text.lower():
            break
        
        query = response.text  # Use the response as the next query
    
    return response.text

In [11]:
question = "What is Valkyria Chronicles?"
answer = rag_with_knowledge_graph(question)
print(f"Question: {question}")
print(f"Answer: {answer}")

Llama.generate: 67 prefix-match hit, remaining 1 prompt tokens to eval

llama_print_timings:        load time =    9485.70 ms
llama_print_timings:      sample time =       4.40 ms /    43 runs   (    0.10 ms per token,  9768.29 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    2507.34 ms /    43 runs   (   58.31 ms per token,    17.15 tokens per second)
llama_print_timings:       total time =    2556.11 ms /    43 tokens

llama_print_timings:        load time =   12741.16 ms
llama_print_timings:      sample time =      14.99 ms /   157 runs   (    0.10 ms per token, 10471.55 tokens per second)
llama_print_timings: prompt eval time =  140800.59 ms /  4662 tokens (   30.20 ms per token,    33.11 tokens per second)
llama_print_timings:        eval time =   10266.89 ms /   156 runs   (   65.81 ms per token,    15.19 tokens per second)
llama_print_timings:    

Question: What is Valkyria Chronicles?
Answer: 



The information provided does not directly answer the question "What is Valkyria Chronicles?" as it only describes Valkyria Chronicles III, which is the third game in the Valkyria Chronicles series. However, based on the context, it can be inferred that Valkyria Chronicles is a tactical role-playing game series. Therefore, I will provide an answer based on this inference:

Valkyria Chronicles is a tactical role-playing game series.

Please note that this answer is based on the provided information and may not be the exact definition of Valkyria Chronicles. A more accurate answer would require additional information about the series. 





If you want to know more about Valkyria Chronicles, you can refer to the provided information or search for more information about the series.
